In [11]:
import pandas as pd

# Load the startup data
df = pd.read_csv('/content/startup_data.csv')

# Convert 'Month' column to datetime objects for proper time-series analysis
df['Month'] = pd.to_datetime(df['Month'], format='%b-%y')

# --- Calculate Burn Rate ---
# Burn rate is the rate at which a company is losing money.
# It is calculated as Monthly Expenses minus Monthly Revenue.
df['Burn_Rate'] = df['Monthly_Expenses'] - df['Monthly_Revenue']
print("Burn Rate calculated per entry:")
print(df[['Month', 'Monthly_Revenue', 'Monthly_Expenses', 'Burn_Rate']].head())

# --- Customer Acquisition Cost (CAC) and Lifetime Value (LTV) ---
# CAC and LTV are directly available in the dataset in the 'CAC' and 'LTV' columns, respectively.
print("\nOriginal CAC values per entry:")
print(df[['Month', 'CAC']].head())
print("\nOriginal LTV values per entry:")
print(df[['Month', 'LTV']].head())

# --- Calculate LTV:CAC Ratio ---
# The LTV:CAC Ratio is calculated by dividing Lifetime Value (LTV) by Customer Acquisition Cost (CAC).
# A check for division by zero is included to prevent errors.
df['LTV_CAC_Ratio'] = df.apply(lambda row: row['LTV'] / row['CAC'] if row['CAC'] != 0 else 0, axis=1)
print("\nLTV:CAC Ratio calculated per entry:")
print(df[['Month', 'LTV', 'CAC', 'LTV_CAC_Ratio']].head())

# --- Aggregate monthly KPIs ---
# To get overall monthly figures, especially if the original data has multiple entries per month
# for different categories, we aggregate by summing up the relevant metrics.
monthly_kpis = df.groupby('Month').agg(
    Monthly_Revenue=('Monthly_Revenue', 'sum'),
    Monthly_Expenses=('Monthly_Expenses', 'sum'),
    Total_CAC=('CAC', 'sum'),
    Total_LTV=('LTV', 'sum'),
    Total_Burn_Rate=('Burn_Rate', 'sum')
).reset_index()

# Re-calculate LTV:CAC Ratio based on the aggregated Total_LTV and Total_CAC for monthly totals
monthly_kpis['LTV_CAC_Ratio'] = monthly_kpis.apply(lambda row: row['Total_LTV'] / row['Total_CAC'] if row['Total_CAC'] != 0 else 0, axis=1)

print("\nAggregated Monthly KPIs:")
print(monthly_kpis.head())

# Optionally, save the aggregated monthly KPIs to a new CSV file
monthly_kpis.to_csv('Cleaned_startup_data.csv', index=False)

Burn Rate calculated per entry:
       Month  Monthly_Revenue  Monthly_Expenses  Burn_Rate
0 2022-01-01         15470.93          16918.48    1447.55
1 2022-01-01         15818.14          16504.86     686.72
2 2022-03-01         15681.24          17076.63    1395.39
3 2022-04-01         15539.42          15850.70     311.28
4 2022-05-01         16015.74          15612.08    -403.66

Original CAC values per entry:
       Month     CAC
0 2022-01-01  251.40
1 2022-01-01  254.21
2 2022-03-01  256.04
3 2022-04-01  250.50
4 2022-05-01  253.67

Original LTV values per entry:
       Month      LTV
0 2022-01-01  4787.11
1 2022-01-01  6254.07
2 2022-03-01  6225.44
3 2022-04-01  6256.58
4 2022-05-01  3412.73

LTV:CAC Ratio calculated per entry:
       Month      LTV     CAC  LTV_CAC_Ratio
0 2022-01-01  4787.11  251.40      19.041806
1 2022-01-01  6254.07  254.21      24.601983
2 2022-03-01  6225.44  256.04      24.314326
3 2022-04-01  6256.58  250.50      24.976367
4 2022-05-01  3412.73  253.67 